yolo v8 nano

Moving images and json into separate folders

In [3]:
import os
import shutil

# Define source directory (CHILLIE)
base_dir = "/home/divya/CHILLIE"

# Define target directories
image_folder = os.path.join(base_dir, "images")
json_folder = os.path.join(base_dir, "json")

# Create target folders if they don't exist
os.makedirs(image_folder, exist_ok=True)
os.makedirs(json_folder, exist_ok=True)

# Loop through folders 1 to 76
for folder_num in range(1, 89):
    folder_path = os.path.join(base_dir, str(folder_num))
    
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        # Copy images and JSON files
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                shutil.copy2(file_path, image_folder)  # Copy images
            elif file.lower().endswith(".json"):
                shutil.copy2(file_path, json_folder)   # Copy JSONs

print("✅ All images and JSON files have been copied successfully!")


✅ All images and JSON files have been copied successfully!


Converting json into YOLO format

In [7]:
import json
import os

# Define paths
json_folder = "/home/divya/CHILLIE/json"
output_folder = "/home/divya/CHILLIE/labels"
os.makedirs(output_folder, exist_ok=True)

# Image dimensions (change if different)
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024

# Process each JSON file
for json_file in os.listdir(json_folder):
    if not json_file.endswith(".json"):
        continue

    json_path = os.path.join(json_folder, json_file)

    try:
        with open(json_path, "r", encoding="utf-8") as file:
            data = json.load(file)
    except Exception as e:
        print(f"❌ Failed to parse {json_file}: {e}")
        continue

    for key, value in data.items():
        if not isinstance(value, dict):
            continue  # Skip non-dict values

        if "filename" not in value or "regions" not in value:
            continue  # Skip invalid entries

        image_name = value["filename"]
        regions = value["regions"]

        if not regions:
            continue  # Skip if no regions

        yolo_filename = os.path.splitext(image_name)[0] + ".txt"
        yolo_filepath = os.path.join(output_folder, yolo_filename)

        with open(yolo_filepath, "w") as yolo_file:
            for region in regions:
                shape = region.get("shape_attributes", {})

                # Optional: skip if not a rectangle
                if shape.get("name") != "rect":
                    continue

                if not all(k in shape for k in ["x", "y", "width", "height"]):
                    continue  # Incomplete shape

                x = shape["x"]
                y = shape["y"]
                w = shape["width"]
                h = shape["height"]

                # Convert to YOLO format
                x_center = (x + w / 2) / IMAGE_WIDTH
                y_center = (y + h / 2) / IMAGE_HEIGHT
                w /= IMAGE_WIDTH
                h /= IMAGE_HEIGHT

                # Write: class_id x_center y_center width height
                yolo_file.write(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("\n✅ All valid JSON files converted to YOLO format in 'CHILLIE/labels/'!")



✅ All valid JSON files converted to YOLO format in 'CHILLIE/labels/'!


Splitting Data into training and validation

In [8]:
import os
import random
import shutil

# Paths
image_folder = "/home/divya/CHILLIE/images"
label_folder = "/home/divya/CHILLIE/labels"
train_image_folder = "/home/divya/CHILLIE/train_image_folder"
val_image_folder = "/home/divya/CHILLIE/val_image_folder"
train_label_folder = "/home/divya/CHILLIE/train_label_folder"
val_label_folder = "/home/divya/CHILLIE/val_label_folder"

# Create directories
os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(val_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

# List images
all_images = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]
random.shuffle(all_images)

# Split 80% train, 20% val
split_index = int(0.8 * len(all_images))
train_images = all_images[:split_index]
val_images = all_images[split_index:]

# Move files
for img in train_images:
    shutil.move(os.path.join(image_folder, img), os.path.join(train_image_folder, img))
    label_file = img.replace(".jpg", ".txt").replace(".png", ".txt")
    if os.path.exists(os.path.join(label_folder, label_file)):
        shutil.move(os.path.join(label_folder, label_file), os.path.join(train_label_folder, label_file))

for img in val_images:
    shutil.move(os.path.join(image_folder, img), os.path.join(val_image_folder, img))
    label_file = img.replace(".jpg", ".txt").replace(".png", ".txt")
    if os.path.exists(os.path.join(label_folder, label_file)):
        shutil.move(os.path.join(label_folder, label_file), os.path.join(val_label_folder, label_file))

print("Dataset split completed!")


Dataset split completed!


In [ ]:
import os
import random
import shutil

# Existing folders (already split into train and val)
train_image_folder = "/home/divya/CHILLIE/train_image_folder"
train_label_folder = "/home/divya/CHILLIE/train_label_folder"
val_image_folder = "/home/divya/CHILLIE/val_image_folder"
val_label_folder = "/home/divya/CHILLIE/val_label_folder"

# New test folders
test_image_folder = "/home/divya/CHILLIE/test_image_folder"
test_label_folder = "/home/divya/CHILLIE/test_label_folder"
os.makedirs(test_image_folder, exist_ok=True)
os.makedirs(test_label_folder, exist_ok=True)

# List images from current training folder
all_train_images = [f for f in os.listdir(train_image_folder) if f.endswith((".jpg", ".png"))]
random.shuffle(all_train_images)

# Move 10–20% of current training data to test
test_size = int(0.2 * len(all_train_images))  # adjust this to 0.1 for 10%
test_images = all_train_images[:test_size]

for img in test_images:
    # Move image
    shutil.move(os.path.join(train_image_folder, img), os.path.join(test_image_folder, img))
    
    # Move label
    label_file = img.replace(".jpg", ".txt").replace(".png", ".txt")
    src_label_path = os.path.join(train_label_folder, label_file)
    dest_label_path = os.path.join(test_label_folder, label_file)

    if os.path.exists(src_label_path):
        shutil.move(src_label_path, dest_label_path)

print("✅ Test set split completed from training data!")


✅ Test set split completed from training data!


In [22]:
import os

# Define the paths
folders = {
    "Train Images": "/home/divya/CHILLIE/images/train",
    "Train Labels": "/home/divya/CHILLIE/labels/train",
    "Val Images": "/home/divya/CHILLIE/images/val",
    "Val Labels": "/home/divya/CHILLIE/labels/val",
    "Test Images": "/home/divya/CHILLIE/images/test",
    "Test Labels": "/home/divya/CHILLIE/labels/test"
}

# Count and display files
print("📊 Dataset Summary:\n")
for name, path in folders.items():
    num_files = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
    print(f"{name:<20}: {num_files} files")


📊 Dataset Summary:

Train Images        : 1671 files
Train Labels        : 1432 files
Val Images          : 522 files
Val Labels          : 463 files
Test Images         : 417 files
Test Labels         : 366 files


In [9]:
pip install ultralytics


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Normalizing coordinates of bounding boxes before passing to YOLO model

In [12]:
import os
import cv2

labels_dir = "/home/divya/CHILLIE/labels/train"
images_dir = "/home/divya/CHILLIE/images/train"

for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)
    image_path = os.path.join(images_dir, label_file.replace('.txt', '.jpg'))  # Adjust extension if needed

    if not os.path.exists(image_path):
        continue  # Skip if image does not exist

    # Get image dimensions
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image: {image_path}")
        continue

    h, w, _ = img.shape  # Image height and width

    with open(label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id, x, y, width, height = map(float, parts)

            # Check if coordinates are absolute (greater than 1 in pixels)
            if x > 1 or y > 1 or width > 1 or height > 1:
                x /= w
                y /= h
                width /= w
                height /= h

            new_lines.append(f"{int(class_id)} {x:.6f} {y:.6f} {width:.6f} {height:.6f}\n")

    with open(label_path, 'w') as f:
        f.writelines(new_lines)

print("Labels normalized successfully!")


Labels normalized successfully!


In [13]:
import os
import cv2

labels_dir = "/home/divya/CHILLIE/labels/val"
images_dir = "/home/divya/CHILLIE/images/val"

for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)
    image_path = os.path.join(images_dir, label_file.replace('.txt', '.jpg'))  # Adjust extension if needed

    if not os.path.exists(image_path):
        continue  # Skip if image does not exist

    # Get image dimensions
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image: {image_path}")
        continue

    h, w, _ = img.shape  # Image height and width

    with open(label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id, x, y, width, height = map(float, parts)

            # Check if coordinates are absolute (greater than 1 in pixels)
            if x > 1 or y > 1 or width > 1 or height > 1:
                x /= w
                y /= h
                width /= w
                height /= h

            new_lines.append(f"{int(class_id)} {x:.6f} {y:.6f} {width:.6f} {height:.6f}\n")

    with open(label_path, 'w') as f:
        f.writelines(new_lines)

print("Labels normalized successfully!")


Labels normalized successfully!


Training YOLOv8 Model

In [ ]:
from ultralytics import YOLO


model = YOLO("yolov8n.pt") 

# Train the model
model.train(
    data="/home/divya/CHILLIE/chillie.yaml", 
    epochs=50, 
    imgsz=640, 
    batch=2,  
    device=1  
)


New https://pypi.org/project/ultralytics/8.3.105 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.100 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/divya/CHILLIE/chillie.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=1, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

E0000 00:00:1744208880.498009    9789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744208880.619538    9789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744208881.594968    9789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744208881.595034    9789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744208881.595039    9789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744208881.595042    9789 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /home/divya/CHILLIE/labels/train... 1432 images, 476 backgrounds, 0 corrupt: 100%|██████████| 1671/1671 [00:01<00:00, 1397.24it/s]

train: New cache created: /home/divya/CHILLIE/labels/train.cache


...

      50/50      1.24G      1.603       2.59      1.615          3        640: 100%|██████████| 836/836 [01:14<00:00, 11.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 131/131 [00:04<00:00, 31.37it/s]


                   all        522       2252      0.275      0.189     0.0928     0.0534

50 epochs completed in 1.078 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.100 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 131/131 [00:03<00:00, 40.51it/s]


                   all        522       2252      0.284      0.193     0.0933     0.0558
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9fee2f86a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

<h3>Hypertuning</h3>

In [ ]:
import torch
from ultralytics import YOLO

# Check available GPUs
num_gpus = torch.cuda.device_count()
print(f"Detected {num_gpus} GPUs")
if num_gpus > 1:
    device = ",".join(str(i) for i in range(num_gpus))  # e.g., "0,1" for 2 GPUs
else:
    device = "0" if num_gpus == 1 else "cpu"  # Fallback to single GPU or CPU

# Load the pretrained model
model = YOLO("yolov8n.pt")

# Run hyperparameter tuning with corrected settings
model.tune(
    data="/home/divya/CHILLIE/chillie.yaml",  # Your dataset config
    epochs=10,  # Fewer epochs for tuning runs
    iterations=15,  # 15 trials
    imgsz=640,  # Consistent with training
    batch=6,  # Adjust if multi-GPU allows more
    device=device,  # Dynamic GPU assignment
    optimize=True,  # Optimize for mAP
    plots=True,  # Visualize progress
    save=True,  # Save best hyperparameters
    project="runs/tune",  # Output directory
    name="tune_chillie_v3",  # New experiment name to avoid overwrite
    cos_lr=True,  # Cosine learning rate scheduler
    # Enhanced augmentation parameters
    hsv_h=0.02,  # Hue variation
    hsv_s=0.7,  
    hsv_v=0.4,  
    flipud=0.5, 
    fliplr=0.5, 
    mosaic=1.0, 
    mixup=0.2,  
    degrees=10.0  
)

print("✅ Hyperparameter tuning with enhanced settings completed!")

Detected 6 GPUs
Tuner: Initialized Tuner instance with 'tune_dir=runs/tune/tune_chillie_v32'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/15 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.02, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 10.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.5, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.2, 'copy_paste': 0.0}


New https://pypi.org/project/ultralytics/8.3.106 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.100 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
                                                        CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
                                                        CUDA:2 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
                                                        CUDA:3 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
                                                        CUDA:4 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
                                                        CUDA:5 (NVIDIA GeForce GTX 1080 Ti, 11172MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/divya/CHILLIE/chillie.yaml, epochs=10, time=None, patience=100, batch=6, imgsz=640, save=True, save_period=-1, cache=False, device=(0, 1, 2, 3, 4, 5), workers=8, project=runs/tune, name=train14, exist_ok=False, pretra

E0000 00:00:1744297580.356869   52966 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744297580.362486   52966 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744297580.376623   52966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297580.376697   52966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297580.376726   52966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297580.376743   52966 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

E0000 00:00:1744297587.596947   52972 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744297587.602595   52972 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744297587.616620   52972 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297587.616704   52972 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297587.616720   52972 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744297587.616732   52972 computation_placer.cc:177] computation placer already registered. Please check linka

...

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/divya/CHILLIE/labels/val.cache... 463 images, 135 backgrounds, 0 corrupt: 100%|██████████| 522/522 [00:00<?, ?it/s]


Plotting labels to runs/tune/train15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01387' and 'momentum=0.98' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0006187499999999999), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 30 dataloader workers
Logging results to runs/tune/train15
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/279 [00:00<?, ?it/s][rank1]: Traceback (most recent call last):
[rank1]:   File "/home/divya/.config/Ultralytics/DDP/_temp_k6zp7tp5136714285071808.py", line 13, in <module>
[rank1]:     results = trainer.train()
[rank1]:   File "/home/divya/.local/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 211, in train
[rank1]:     self._do_train(world_size)
[rank1]:   File "/home/divya/.local/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 385, in _do_train
[rank1]:     loss, self.loss_items = self.model(batch)
[rank1]:   File "/home/divya/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
[rank1]:     return self._call_impl(*args, **kwargs)
[rank1]:   File "/home/divya/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
[rank1]:     return forward_call(*args, **kwargs)
[rank1]:   File "/home/divya/.local/lib/python3.10/site-packages/torch/nn/parallel/distri

KeyboardInterrupt: 

In [1]:
import torch
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")


Allocated: 0.00 MB
Reserved: 0.00 MB


In [4]:
free, total = torch.cuda.mem_get_info()
print(f"Free: {free / 1024**3:.2f} GB")
print(f"Total: {total / 1024**3:.2f} GB")


Free: 10.64 GB
Total: 10.91 GB


In [3]:
torch.cuda.empty_cache()
